In [5]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import acquire
import prepare

In [6]:
rawdf = acquire.get_zillow_data()

In [7]:
df = prepare.prep_zillow(rawdf)

In [8]:
train, validate, test = prepare.split(df)

In [9]:
train.columns

Index(['location', 'bedrooms', 'bathrooms', 'square_feet', 'lot_size',
       'year_built', 'assessed_value', 'tax_amount', 'county'],
      dtype='object')

In [10]:
predictions = pd.DataFrame({
    'actual': validate.assessed_value
}) 

In [11]:
predictions.head()

,actual
41123,546433.0
32314,75342.0
46201,355737.0
8569,104772.0
50247,278050.0


## Simple Model
$$ y = mx + b $$

$$y = f(x)$$

In [13]:
from sklearn.linear_model import LinearRegression

# X must be 2-d
X_train = train[['bedrooms', 'bathrooms', 'square_feet', 'lot_size', 'year_built', 'county']]
# y can be 1-d
y_train = train.assessed_value

# 1. make the thing
lm = LinearRegression()
# 2. fit the thing
lm.fit(X_train, y_train)
# 3. use the thing (make predictions)
# X_validate = validate[['location', 'bedrooms', 'bathrooms', 'square_feet', 'lot_size', 'year_built', 'county']]
predictions['simple_lm'] = lm.predict(X_train)

ValueError: could not convert string to float: 'Los Angeles'